## **앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


개별 분류기의 최빈값(다수결)으로 최종 예측을 정하는 것을 **직접 투표** 분류기,

개별 분류기의 클래스 확률을 평균한 뒤 가장 높은 확률의 클래스를 선택하는 것을 **간접 투표** 라고 함.

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

learning_rate 를 낮게 설정해 규제를 활용해 과대 적합을 해소시킴.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [2]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [7]:
# train/valid/test dataset
y = y.astype(int) # 정수형으로 변환

from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=10000, random_state=42, stratify=y
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_temp, y_temp, test_size=10000, random_state=42, stratify=y_temp
)


####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다.

In [4]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [9]:
# model fitting

rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(C=1.0, random_state=42)
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,),
                        activation='relu',
                        max_iter=20,
                        random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [10]:
# model fitting

rnd_clf.fit(X_train, y_train)
et_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)
mlp_clf.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=20, random_state=42)

In [11]:
# model test
from sklearn.ensemble import VotingClassifier

# 직접 투표 앙상블 (hard voting)
voting_clf = VotingClassifier(
    estimators=[('rf', rnd_clf), ('et', et_clf), ('svm', svm_clf), ('mlp', mlp_clf)],
    voting='hard'   # soft는 predict_proba 필요, LinearSVC는 지원X → hard로
)

voting_clf.fit(X_train, y_train)

print("\nVotingClassifier (hard) 검증 정확도")
print("Voting :", voting_clf.score(X_valid, y_valid))


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(



VotingClassifier (hard) 검증 정확도
Voting : 0.97


####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [12]:
# 각 분류 모델 학습

# 개별 모델 성능 확인
models = [
    ("rf", rnd_clf),
    ("et", et_clf),
    ("svm", svm_clf),
    ("mlp", mlp_clf)
]


In [13]:
# 각 분류 모델의 성능 확인
scores = {}
for name, model in models:
    acc = model.score(X_valid, y_valid)
    scores[name] = acc
    print(f"{name} 검증 정확도: {acc:.4f}")


rf 검증 정확도: 0.9709
et 검증 정확도: 0.9731
svm 검증 정확도: 0.9140
mlp 검증 정확도: 0.9492


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. svm

In [14]:
# 가장 성능이 낮은 모델 제거
worst_model = min(scores, key=scores.get)
print("\n가장 성능이 낮은 모델:", worst_model)

# 가장 성능이 낮은 모델 제거 후 앙상블
models_reduced = [(name, model) for name, model in models if name != worst_model]
voting_reduced = VotingClassifier(estimators=models_reduced, voting="hard")



가장 성능이 낮은 모델: svm


In [15]:
# model fitting
voting_reduced.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('et', ExtraTreesClassifier(random_state=42)),
                             ('mlp',
                              MLPClassifier(max_iter=20, random_state=42))])

In [16]:
# 모델 제거 후 성능 확인
print(f"'{worst_model}' 제거 후 Voting 정확도:", voting_reduced.score(X_valid, y_valid))


'svm' 제거 후 Voting 정확도: 0.9731


### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [17]:
# (1) 각 기저 모델의 검증셋 예측 (라벨 기반; 모든 모델에서 공통으로 사용 가능)
valid_rf  = rnd_clf.predict(X_valid)
valid_et  = et_clf.predict(X_valid)
valid_svm = svm_clf.predict(X_valid)
valid_mlp = mlp_clf.predict(X_valid)

# (2) 메타 특성으로 결합 (shape: [len(X_valid), 4])
X_valid_meta = np.c_[valid_rf, valid_et, valid_svm, valid_mlp]
y_valid_meta = y_valid  # 타깃은 검증셋의 정답 라벨

print("X_valid_meta shape:", X_valid_meta.shape)
print("예시 행:", X_valid_meta[:5])

X_valid_meta shape: (10000, 4)
예시 행: [[3 3 3 3]
 [3 3 3 3]
 [4 4 4 4]
 [0 0 0 0]
 [3 3 3 3]]


####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [18]:
from sklearn.ensemble import RandomForestClassifier

blender = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)
blender.fit(X_valid_meta, y_valid_meta)
print("블렌더 학습 완료")

블렌더 학습 완료


- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다.

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [19]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
test_rf  = rnd_clf.predict(X_test)
test_et  = et_clf.predict(X_test)
test_svm = svm_clf.predict(X_test)
test_mlp = mlp_clf.predict(X_test)

In [20]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
X_test_meta = np.c_[test_rf, test_et, test_svm, test_mlp]

In [22]:
# model test
from sklearn.metrics import accuracy_score

test_pred_stack = blender.predict(X_test_meta)
acc_stack = accuracy_score(y_test, test_pred_stack)
print(f"스태킹 앙상블 Test 정확도: {acc_stack:.4f}")

스태킹 앙상블 Test 정확도: 0.9659
